In [27]:
from fastapi import FastAPI
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import pickle

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Ari
[nltk_data]     Widyatama\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [28]:
app = FastAPI()

In [29]:
#inialization stopWords and Steamer
stopWords = stopwords.words('indonesian')

factory = StemmerFactory()
stemmer = factory.create_stemmer()


Import data and Model

In [30]:
with open('./data/dataset.pkl', 'rb') as file:
    patterns = pickle.load(file)

In [31]:
modelChatbot = tf.keras.models.load_model('./model/chatbotModel.h5')

In [32]:
modelClass = tf.keras.models.load_model('./model/classfication.h5')

In [33]:
with open('./data/responses.pkl','rb') as file:
    responses = pickle.load(file)

In [34]:
with open('./data/labelVocab.pkl', 'rb') as file:
    labelVocab = pickle.load(file)


In [35]:
#standardize text to all lowercase letters, cut stopwords and symbols
def standardizeFunc(sentence):
    stopwords = stopWords
    
    sentence = tf.strings.lower(sentence)

    for word in stopwords:
        if word[0] == "'":
            sentence = tf.strings.regex_replace(sentence, rf"{word}\b", "")
        else:
            sentence = tf.strings.regex_replace(sentence, rf"\b{word}\b", "")
    sentence = tf.strings.regex_replace(sentence, r'[!"#$%&()\*\+,-\./:;<=>?@\[\\\]^_`{|}~\']', "")

    return sentence

In [36]:
#create a vectorizer function for the text that will be input to predict the model
maxLeght = 30
def fitVectorizer(trainSentences, standardizeFunc):
  vectorizer = tf.keras.layers.TextVectorization(
    standardize=standardizeFunc,
    output_sequence_length=maxLeght
  )

  vectorizer.adapt(trainSentences)
  return vectorizer

In [37]:
#create a vertoriser by providing question data to the train for adapt.
vectorizer = fitVectorizer(patterns, standardizeFunc)

In [38]:
vocabSize = vectorizer.vocabulary_size()
print(vocabSize)

204


In [39]:
#retrieves data to be adapted to the labelEncoder.
labelEncoder = tf.keras.layers.StringLookup(num_oov_indices=0)
labelEncoder.set_vocabulary(labelVocab)

print(f'Loaded vocabulary: {labelEncoder.get_vocabulary()}')

Loaded vocabulary: ['pahit_di_mulut', 'menular', 'mengelola_stres', 'genetik', 'air_dingin', 'soda', 'mengobati', 'makanan_pemicu', 'faktor_umur', 'bahaya', 'tidur_dan_berbaring_setelah_makan', 'sesak_napas', 'pengaruh_asam_lambung_pada_kehidupan', 'mual', 'minumam_beresiko', 'kopi', 'kematian', 'gejala', 'alkohol', 'rekomendasi_buah', 'perut_kembung', 'penyakit_turunan', 'panas_di_dada', 'olahraga', 'merokok', 'kanker', 'waktu_makan_terbaik', 'thanks', 'sembuh', 'pola_hidup_pengaruh_asam_lambung', 'penyebab', 'pedas', 'makanan_penunjang', 'makan tidak teratur', 'definisi', 'berat_badan', 'bahaya_dari_diet', 'nasi_putih', 'menghindari', 'kentang_goreng', 'kambuh', 'diagnosa', 'cemilan_sehat', 'bersendawa', 'asam', 'tinggi_badan', 'tanda_parah', 'sulit_menelan', 'stress', 'sakit_tenggorokan', 'pychan', 'mengatasi_rasa_perih_di_ulu_hati', 'makanan_berminyak', 'madu_meredakan', 'maag', 'konsumsi_obat', 'kondisi_kritis', 'jenis_kelamin', 'hidup_sehat_untuk_penderita', 'greeting', 'diet_veg

In [40]:
#determine the minimum confidence and provide response results based on tags
def getResponse(predictTag, confidence):
    if confidence < 0.55:
        return "Maaf, input yang Anda masukkan kurang jelas."
    return np.random.choice(responses[predictTag])

In [41]:
#create functions that accept input for preprocessing, prediction, and return response
def showRespon(input):
    stemming = stemmer.stem(input)
    inputA = vectorizer([stemming])
    prediction = modelChatbot.predict(inputA)

    predictClassIndex = np.argmax(prediction, axis=-1)
    confidence = prediction[0][predictClassIndex[0]]
    predictTag = labelEncoder.get_vocabulary()[predictClassIndex[0]]
    response = getResponse(predictTag, confidence)

    return response


In [42]:
testInput = "apakah sering konsumsi obat dapat memperparah asam lambung"

response = showRespon(testInput)

print(f"Response: {response}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 588ms/step
Response: Banyak jenis obat memiliki efek samping yang dapat mengganggu sistem pencernaan, termasuk meningkatkan produksi asam lambung atau merusak lapisan pelindung lambung.

1. Obat antiinflamasi nonsteroid (OAINS): Obat seperti ibuprofen, naproxen, dan aspirin sering digunakan untuk meredakan nyeri dan peradangan. Namun, obat-obatan ini dapat mengiritasi lapisan lambung dan meningkatkan produksi asam lambung.

2. Aspirin: Aspirin dosis rendah sering digunakan untuk mencegah serangan jantung dan stroke. Namun, aspirin juga dapat meningkatkan risiko perdarahan lambung dan iritasi.

3. Kortikosteroid: Obat ini sering digunakan untuk mengobati peradangan dan penyakit autoimun. Namun, penggunaan jangka panjang kortikosteroid dapat meningkatkan risiko osteoporosis, meningkatkan tekanan darah, dan juga dapat mengganggu produksi asam lambung.

4. Bisfosfonat: Obat ini digunakan untuk mengobati osteoporosis. Beberapa orang yang mengonsumsi bisfosfonat m

In [43]:
#function to get prediction result based on input
def predictAsamlambung(input_data):
    if isinstance(input_data, dict):
        input_data = pd.DataFrame([input_data])  
    
    prediction = modelClass.predict(input_data)
    predictedPercentage = prediction[0][0]
    
    if predictedPercentage >= 0.5:
        predictedlabel = "Ya"
        lifestyle = (
            "Anda terdeteksi terkena Asam Lambung, anda dapat berkonsultasi lebih lanjut melalui Chatbot!\n"
            "Terapkan Pola Hidup Sehat berikut ini untuk meredakan gejala Asam Lambung:\n\n"

            "1. Menurunkan Berat Badan\n"
            "Obesitas adalah salah satu penyebab utama GERD. Lemak perut berlebih memberikan tekanan pada lambung, sehingga mendorong asam lambung naik ke kerongkongan.\n"
            "- Terapkan pola makan sehat dengan rendah lemak dan kalori.\n"
            "- Rutin berolahraga untuk menjaga berat badan ideal.\n\n"
            
            "2. Hindari Makanan Pemicu Refluks\n"
            "Banyak sekali makanan yang mudah memicu refluks asam. Jika kamu mengidap penyakit asam lambung, sebaiknya hindari jenis makanan berikut:\n"
            "- Makanan berlemak dan pedas.\n"
            "- Makanan asam (jeruk, tomat).\n"
            "- Minuman berkafein atau berkarbonasi (kopi, soda).\n"
            "- Cokelat, daun mint, dan bawang.\n"
            "Hindari atau kurangi konsumsi makanan tersebut untuk mencegah gejala memburuk.\n\n"

            "3. Makan dalam Porsi Kecil\n"
            "Makan dalam porsi besar dapat memberikan tekanan lebih pada otot kerongkongan bawah dan memicu refluks asam. Jadi, makanlah dalam porsi kecil.\n\n"

            "4. Jangan Berbaring Setelah Makan\n"
            "Berbaring setelah makan sering menjadi kebiasaan yang banyak orang tidak sadari. Padahal, kebiasaan yang satu ini menjadi salah satu penyebab utama naiknya asam lambung.\n"
            "Tunggu minimal 2–3 jam sebelum berbaring atau tidur.\n\n"

            "5. Jangan Merokok\n"
            "Beberapa penelitian menemukan bahwa nikotin dapat mengendurkan otot-otot kerongkongan bagian bawah dan juga dapat mengganggu kemampuan air liur untuk membersihkan asam dari kerongkongan.\n\n"

            "6. Hindari Alkohol\n"
            "Alkohol dapat meningkatkan produksi asam lambung dan mengiritasi kerongkongan. Batasi atau hindari konsumsi alkohol untuk mengelola gejala.\n\n"

            "7. Perhatikan Obat yang Dikonsumsi\n"
            "Beberapa obat ini mampu melemaskan otot kerongkongan bagian bawah, mengganggu proses pencernaan, atau mengiritasi kerongkongan yang sudah meradang. Obat-obatan ini seperti:\n"
            "- Obat antiinflamasi non-steroid.\n"
            "- Penghambat saluran kalsium untuk hipertensi.\n"
            "- Obat asma tertentu, antibiotik, atau zat besi.\n"
            "Jika sedang mengonsumsi obat ini, konsultasikan dengan dokter agar lebih aman.\n\n"

            "8. Kenakan Pakaian Longgar\n"
            "Hindari pakaian ketat atau ikat pinggang yang menekan perut karena dapat memicu naiknya asam lambung."
        )
    else:
        predictedlabel = "Tidak"
        lifestyle = (
            "Anda terdeteksi tidak terkena Asam Lambung, jika ada pertanyaan lebih lanjut, silahkan konsultasikan melalui Chatbot!\n"
            "Untuk Selalu terhindar dari Asam Lambung, Anda dapat menjaga lambung agar tetap sehat dengan menerapkan pola hidup sehat berikut ini:\n\n"
            
            "1. Cukupi Asupan Cairan\n"
            "Minumlah air putih dalam jumlah yang cukup setiap hari untuk membantu proses pencernaan dan menjaga lambung tetap sehat. Disarankan mengonsumsi 6-8 gelas air putih setiap hari.\n\n"
            
            "2. Makan Tepat Waktu\n"
            "Makanlah tepat waktu agar lambung senantiasa sehat. Makan dengan jadwal yang tidak teratur, bisa meningkatkan refluks asam lambung, hingga memicu gejala maag pada pengidapnya. Oleh sebab itu, konsumsi makanan dan camilan sehat secara teratur. Lalu, usahakan untuk duduk saat makan, dan makan pada waktu yang sama setiap harinya.\n\n"
            
            "3. Sempatkan waktu untuk berolahraga\n"
            "Olahraga tidak hanya dapat memperkuat tulang dan otot-otot tubuh saja. Aktivitas fisik yang teratur juga bisa menjaga makanan tetap bergerak melalui sistem pencernaan, sehingga dapat mencegah sembelit serta masalah pencernaan lainnya. Di samping itu, olahraga juga bisa mempertahankan berat badan yang ideal yang baik untuk sistem pencernaan.\n\n"
            
            "4. Hindari Asupan Alkohol dan Rokok\n"
            "Hindari kebiasaan mengonsumsi alkohol dan merokok. Dari kedua kebiasaan ini bisa mengganggu fungsi lambung dan sistem pencernaan secara keseluruhan.\n\n"
            
            "5. Kurangi Makanan Berlemak\n"
            "Makanan berlemak dapat memperlambat proses pencernaan dan meningkatkan risiko gangguan lambung. Pilih makanan dengan kandungan lemak sehat seperti alpukat atau kacang-kacangan, tetapi tetap dalam porsi yang moderat.\n\n"
            
            "6. Makan makanan yang mengandung Serat\n"
            "Serat berperan penting dalam menjaga kesehatan saluran pencernaan, termasuk kesehatan lambung. Kamu bisa memperoleh asupan serat dari buah-buahan, sayuran, kacang-kacangan, dan biji-bijian.\n\n"
            
            "7. Hindari Konsumsi Makanan Pedas dan Asam Berlebihan\n"
            "Walaupun makanan pedas dan asam menggugah selera, konsumsinya secara berlebihan dapat mengiritasi lambung. Batasi konsumsi makanan ini untuk menjaga keseimbangan sistem pencernaan.\n\n"
            
            "8. Porsi Makan yang Seimbang\n"
            "Hindari makan berlebihan dalam satu waktu. Lebih baik makan dengan porsi kecil namun sering, sehingga lambung tidak bekerja terlalu keras."
        )
    
    return predictedlabel, lifestyle

In [44]:
#function to match the input data with the data used to train the previous model
def userInput(umur,bb,rokok,polaMakan,makanPedas,minumSoda,tidurMakan,olahraga,stres,obat,makanLemak,minumKopi,minumAlkohol,
            mual,perutKembung,sakitTenggorokan,kesulitanMenelan,tidakAda,sendawa,panasDada,nyeriPerut):
    inputData = {
        'umur': umur,
        'bb': bb, #Berat Badan
        'merokok': rokok, #Tidak = 0, Iya = 1
        'polaMakan': polaMakan, #Tidak = 0, Iya =1
        'makanPedas': makanPedas, #Skala 1 - 5
        'minumSoda': minumSoda, #Skala 1 - 5
        'tidurSetelahMakan': tidurMakan, #Skala 1 - 5
        'olahraga': olahraga, #Skala 1 - 5
        'strees': stres, #Skala 1 - 5
        'obatObatan': obat, #Skala 1 - 5
        'makanLemak': makanLemak, #Skala 1 - 5
        'minumKopi': minumKopi, #Skala 1 - 5
        'minumAlkohol':minumAlkohol, #Skala 1 - 5
        'Merasa mual jika Stress': mual, #Ada = 1, Tidak Ada = 0
        'Perut kembung': perutKembung, #Ada = 1, Tidak Ada = 0
        'Sakit tenggorokan atau suara serak': sakitTenggorokan, #Ada = 1, Tidak Ada = 0
        'Kesulitan menelan makanan': kesulitanMenelan, #Ada = 1, Tidak Ada = 0
        'Tidak ada':tidakAda, #Ada = 1, Tidak Ada = 0
        'Sering Bersendawa':sendawa, #Ada = 1, Tidak Ada = 0
        'Rasa panas di dada setelah makan atau saat berbaring':panasDada, #Ada = 1, Tidak Ada = 0
        'Nyeri di bagian atas perut atau dada':nyeriPerut #Ada = 1, Tidak Ada = 0
    }
    predictedlabel, lifestyle = predictAsamlambung(inputData)

    return  predictedlabel, lifestyle


Test predict

In [45]:
dataDF = pd.read_csv("./datasets/testAsamLambung.csv")
dataDF

,umur,bb,merokok,polaMakan,makanPedas,minumSoda,tidurSetelahMakan,olahraga,strees,obatObatan,...,minumKopi,minumAlkohol,Merasa mual jika Stress,Sering Bersendawa,Perut kembung,Nyeri di bagian atas perut atau dada,Tidak ada,Kesulitan menelan makanan,Sakit tenggorokan atau suara serak,Rasa panas di dada setelah makan atau saat berbaring
0,20,67,0,1,5,2,4,2,3,2,...,2,1,0,0,1,0,0,0,0,0
1,22,52,0,0,1,3,1,2,5,1,...,1,1,0,0,0,0,1,0,0,0
2,21,54,0,0,3,2,2,3,3,2,...,2,1,0,0,1,0,0,0,1,0
3,19,40,0,1,5,3,5,2,5,5,...,2,1,1,1,0,1,0,1,0,0
4,20,48,0,0,1,2,2,3,2,2,...,3,1,0,0,1,0,0,0,0,0


In [46]:
dataList = dataDF.values.tolist()

for i in dataList:
    predictedlabel, lifestyle = userInput(*i)

    print(f"Predicted Asam Lambung: {predictedlabel}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
Predicted Asam Lambung: Ya
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step
Predicted Asam Lambung: Tidak
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
Predicted Asam Lambung: Ya
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
Predicted Asam Lambung: Ya
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
Predicted Asam Lambung: Ya


In [47]:
predictedlabel, lifestyle = userInput(21,53,1,0,2,4,1,3,1,2,1,1,1,0,0,0,0,1,0,0,0)

print(f"Predicted Asam Lambung: {predictedlabel}")
print(lifestyle)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Predicted Asam Lambung: Tidak
Anda terdeteksi tidak terkena Asam Lambung, jika ada pertanyaan lebih lanjut, silahkan konsultasikan melalui Chatbot!
Untuk Selalu terhindar dari Asam Lambung, Anda dapat menjaga lambung agar tetap sehat dengan menerapkan pola hidup sehat berikut ini:

1. Cukupi Asupan Cairan
Minumlah air putih dalam jumlah yang cukup setiap hari untuk membantu proses pencernaan dan menjaga lambung tetap sehat. Disarankan mengonsumsi 6-8 gelas air putih setiap hari.

2. Makan Tepat Waktu
Makanlah tepat waktu agar lambung senantiasa sehat. Makan dengan jadwal yang tidak teratur, bisa meningkatkan refluks asam lambung, hingga memicu gejala maag pada pengidapnya. Oleh sebab itu, konsumsi makanan dan camilan sehat secara teratur. Lalu, usahakan untuk duduk saat makan, dan makan pada waktu yang sama setiap harinya.

3. Sempatkan waktu untuk berolahraga
Olahraga tidak hanya dapat memperkuat tulang dan otot-otot tubuh saja. Aktivitas fisik ya

In [48]:
#API Chatbot
@app.post("/show_respon/")
async def get_show_respon(input: str):
    response = showRespon(input)
    return {"response": response}

In [49]:
#API Classificaton
@app.post("/user_input/")
async def get_user_input(umur: int, bb: int, rokok: int, polaMakan: int, makanPedas: int, minumSoda: int,
                          tidurMakan: int, olahraga: int, stres: int, obat: int, makanLemak: int, minumKopi: int,
                          minumAlkohol: int, mual: int, perutKembung: int, sakitTenggorokan: int, kesulitanMenelan: int,
                          tidakAda: int, sendawa: int, panasDada: int, nyeriPerut: int):
    predictedlabel, lifestyle = userInput(umur, bb, rokok, polaMakan, makanPedas, minumSoda, tidurMakan, olahraga, stres, obat,
                               makanLemak, minumKopi, minumAlkohol, mual, perutKembung, sakitTenggorokan, kesulitanMenelan,
                               tidakAda, sendawa, panasDada, nyeriPerut)
    return {
        "predicted_label": predictedlabel,
        "lifestyle": lifestyle
    }